In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving spark_nlp_for_healthcare.json to spark_nlp_for_healthcare.json


In [ ]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

! pip install spark-nlp-display

--2021-08-12 11:16:03--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1872 (1.8K) [text/plain]
Saving to: ‘jsl_colab_setup.sh’

jsl_colab_setup.sh  100%[===================>]   1.83K  --.-KB/s    in 0s      

2021-08-12 11:16:03 (28.5 MB/s) - ‘jsl_colab_setup.sh’ saved [1872/1872]

setup Colab for PySpark 3.1.1 and Spark NLP 3.1.3
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/

In [ ]:
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.1.1
Spark NLP_JSL Version : 3.1.3


In [ ]:
with open('nlp_test1 (1).txt', 'r') as file:
    sample = file.read()
    
sample_df = spark.createDataFrame([[sample]]).toDF("text")

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel\
      .pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
      .setInputCols(["document"])\
      .setOutputCol("sentences")

tokenizer = Tokenizer()\
      .setInputCols(["sentences"])\
      .setOutputCol("uncheckedtoken")\

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("uncheckedtoken")\
    .setOutputCol("tokens")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentences", "tokens"])\
      .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models")\
      .setInputCols("sentences", "tokens")\
      .setOutputCol("pos_tags")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
      .setInputCols("sentences", "pos_tags", "tokens")\
      .setOutputCol("dependencies")

clinical_ner_tagger = MedicalNerModel.pretrained("ner_jsl","en","clinical/models")\
      .setInputCols("sentences", "tokens", "embeddings")\
      .setOutputCol("ner_tags")

ner_chunker = NerConverter()\
      .setInputCols("sentences", "tokens", "ner_tags")\
      .setOutputCol("ner_chunks")

relationPairs = ["symptom-relativedate",
                 "relativedate-symptom",
                 "problem-relativedate",
                 "relativedate-problem",
                 "test-date",
                 "date-test"]

re_ner_chunk_filter = RENerChunksFilter()\
        .setInputCols("ner_chunks", "dependencies")\
        .setOutputCol("re_ner_chunks")\
        .setRelationPairs(relationPairs)

re_model = RelationExtractionDLModel.pretrained("redl_clinical_biobert", "en", "clinical/models")\
        .setInputCols("re_ner_chunks", "sentences")\
        .setOutputCol("relations")
 
trained_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        spellModel,
        word_embeddings,
        pos_tagger,
        clinical_ner_tagger,
        ner_chunker,
        dependency_parser,
        re_ner_chunk_filter,
        re_model])

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 363.9 KB
[OK!]
spellcheck_clinical download started this may take some time.
Approximate size to download 142.2 MB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
redl_clinical_biobert download started this may take some time.
Approximate size to download 383.4 MB
[OK!]


In [ ]:
result_sample = trained_pipeline.fit(sample_df).transform(sample_df)

In [ ]:
result=result_sample.selectExpr("explode(relations) as relations")\
 .select(
   "relations.metadata.chunk1",
   "relations.metadata.entity1",
   "relations.metadata.chunk2",
   "relations.metadata.entity2",
   "relations.result"
 ).toPandas()

In [ ]:
for ind in result.index:
  if(result['entity1'][ind]=='RelativeDate' or result['entity1'][ind]=='Date'):
    temp=result['entity1'][ind]
    result['entity1'][ind]=result['entity2'][ind]
    result['entity2'][ind]=temp
    temp=result['chunk1'][ind]
    result['chunk1'][ind]=result['chunk2'][ind]
    result['chunk2'][ind]=temp

In [ ]:
result

,chunk1,entity1,chunk2,entity2,result
0,non-rejuvenating sleep,Symptom,Today's,RelativeDate,O
1,snoring,Symptom,Today's,RelativeDate,O
2,non-rejuvenating sleep,Symptom,3 years ago,RelativeDate,O
3,snoring,Symptom,3 years ago,RelativeDate,O
4,ejection fraction,Test,1993,Date,O
5,pain,Symptom,in the last 6 months,RelativeDate,PIP
6,headaches,Symptom,in the past 2 months,RelativeDate,O
7,blurred vision,Symptom,in the past 2 months,RelativeDate,PIP
8,Doppler Study,Test,1/2/2021,Date,O
9,Doppler Study,Test,12/20/2020,Date,O


In [ ]:
df=spark.createDataFrame(result.astype(str))

In [ ]:
document_assembler = DocumentAssembler().setInputCol('chunk2').setOutputCol('document')
chunksDF = document_assembler.transform(df)
aa = map_annotations_col(chunksDF.select("document"),lambda x: [Annotation('chunk', a.begin, a.end, a.result, a.metadata, a.embeddings) for a in x], "document","chunk_date", "chunk")
dateNormalizer = DateNormalizer().setInputCols('chunk_date').setOutputCol('date').setAnchorDateYear(2021).setAnchorDateMonth(2).setAnchorDateDay(27)
dateDf = dateNormalizer.transform(aa)
dateDf.select("chunk_date.result","date.result").show()

+--------------------+--------------------+
|              result|              result|
+--------------------+--------------------+
|           [Today's]|        [2021/02/27]|
|           [Today's]|        [2021/02/27]|
|       [3 years ago]|       [3 years ago]|
|       [3 years ago]|       [3 years ago]|
|              [1993]|        [1993/MM/DD]|
|[in the last 6 mo...|[in the last 6 mo...|
|[in the past 2 mo...|[in the past 2 mo...|
|[in the past 2 mo...|[in the past 2 mo...|
|          [1/2/2021]|        [2021/01/02]|
|        [12/20/2020]|        [2020/12/20]|
|          [5/2/2020]|        [2020/05/02]|
+--------------------+--------------------+

